In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv


In [2]:
df = pd.read_csv('/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv')

In [3]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers


In [81]:
model = Sequential()
model.add(Dense(units=64,activation='relu', input_dim= 15))
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',optimizer='sgd', metrics=['accuracy'])

In [6]:
y = df["TenYearCHD"]
y

0       0
1       0
2       0
3       1
4       0
       ..
4233    1
4234    0
4235    0
4236    0
4237    0
Name: TenYearCHD, Length: 4238, dtype: int64

In [7]:

X = df[df.columns[:-1]]

X

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN


In [8]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
transformed_values = imputer.fit_transform(X)
transformed_values

array([[  1.        ,  39.        ,   4.        , ...,  26.97      ,
         80.        ,  77.        ],
       [  0.        ,  46.        ,   2.        , ...,  28.73      ,
         95.        ,  76.        ],
       [  1.        ,  48.        ,   1.        , ...,  25.34      ,
         75.        ,  70.        ],
       ...,
       [  0.        ,  48.        ,   2.        , ...,  22.        ,
         84.        ,  86.        ],
       [  0.        ,  44.        ,   1.        , ...,  19.16      ,
         86.        ,  81.96675325],
       [  0.        ,  52.        ,   2.        , ...,  21.47      ,
         80.        , 107.        ]])

In [64]:
for j in range(15):
    min_x = min(transformed_values[:,j])
    max_x = max(transformed_values[:,j])
    for i in range(len(transformed_values[:,j])):
        
        transformed_values[:,j][i] = (transformed_values[:,j][i] - min_x) / (max_x - min_x)
transformed_values

array([[1.        , 0.        , 1.        , ..., 0.27702375, 0.36363636,
        0.10451977],
       [0.        , 0.62816008, 0.18518519, ..., 0.31968008, 0.51515152,
        0.10169492],
       [1.        , 0.65712171, 0.        , ..., 0.23751818, 0.31313131,
        0.08474576],
       ...,
       [0.        , 0.92163004, 1.        , ..., 0.1565681 , 0.4040404 ,
        0.1299435 ],
       [0.        , 0.84326008, 0.32777778, ..., 0.08773631, 0.42424242,
        0.11855015],
       [0.        , 1.        , 1.        , ..., 0.14372273, 0.36363636,
        0.18926554]])

In [66]:
from sklearn.model_selection import train_test_split
X_train,x_test,y_train,y_test = train_test_split(transformed_values,y, test_size = 0.25, random_state = 42, shuffle = True)

In [82]:
model.fit(X_train,y_train, epochs = 300)

Epoch 1/300
3178/3178 [==============================] - 0s 97us/step - loss: 0.5569 - accuracy: 0.8427
Epoch 2/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.4661 - accuracy: 0.8452
Epoch 3/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.4499 - accuracy: 0.8452
Epoch 4/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.4450 - accuracy: 0.8452
Epoch 5/300
3178/3178 [==============================] - 0s 55us/step - loss: 0.4418 - accuracy: 0.8452
Epoch 6/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.4391 - accuracy: 0.8452
Epoch 7/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.4368 - accuracy: 0.8452
Epoch 8/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.4344 - accuracy: 0.8452
Epoch 9/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.4324 - accuracy: 0.8452
Epoch 10/300
3178/3178 [==============================] - 0s 53u

3178/3178 [==============================] - 0s 53us/step - loss: 0.3990 - accuracy: 0.8464
Epoch 80/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3986 - accuracy: 0.8464
Epoch 81/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.3985 - accuracy: 0.8468
Epoch 82/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3982 - accuracy: 0.8468
Epoch 83/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.3979 - accuracy: 0.8468
Epoch 84/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.3977 - accuracy: 0.8468
Epoch 85/300
3178/3178 [==============================] - 0s 57us/step - loss: 0.3975 - accuracy: 0.8468
Epoch 86/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3969 - accuracy: 0.8468
Epoch 87/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3968 - accuracy: 0.8468
Epoch 88/300
3178/3178 [==============================] - 0s 52us/st

3178/3178 [==============================] - 0s 53us/step - loss: 0.3837 - accuracy: 0.8515
Epoch 157/300
3178/3178 [==============================] - 0s 51us/step - loss: 0.3835 - accuracy: 0.8502
Epoch 158/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3834 - accuracy: 0.8502
Epoch 159/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3826 - accuracy: 0.8499
Epoch 160/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3832 - accuracy: 0.8508
Epoch 161/300
3178/3178 [==============================] - 0s 54us/step - loss: 0.3831 - accuracy: 0.8518
Epoch 162/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3829 - accuracy: 0.8505
Epoch 163/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3831 - accuracy: 0.8508
Epoch 164/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3829 - accuracy: 0.8505
Epoch 165/300
3178/3178 [==============================] - 0

3178/3178 [==============================] - 0s 52us/step - loss: 0.3797 - accuracy: 0.8527
Epoch 234/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3792 - accuracy: 0.8531
Epoch 235/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3792 - accuracy: 0.8515
Epoch 236/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3795 - accuracy: 0.8524
Epoch 237/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3792 - accuracy: 0.8537
Epoch 238/300
3178/3178 [==============================] - 0s 52us/step - loss: 0.3790 - accuracy: 0.8527
Epoch 239/300
3178/3178 [==============================] - 0s 51us/step - loss: 0.3778 - accuracy: 0.8537
Epoch 240/300
3178/3178 [==============================] - 0s 53us/step - loss: 0.3791 - accuracy: 0.8531
Epoch 241/300
3178/3178 [==============================] - 0s 51us/step - loss: 0.3792 - accuracy: 0.8540
Epoch 242/300
3178/3178 [==============================] - 0

In [83]:
_,acc = model.evaluate(x_test,y_test)
print("Acc:{}",acc*100)

1060/1060 [==============================] - 0s 98us/step
Acc:{} 86.41509413719177
